In [73]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

root_dir='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/scripts_hongli/'
os.chdir(root_dir)
output_folder='step8_plot_GMET_NLDAS_PT'
outputfile = 'GMET_NLDAS_PT.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

time_format='%Y-%m-%d %H:%M:%S'
# formatter = DateFormatter('%m/%d/%y')
formatter = DateFormatter('%b-%y') 
dpi_value = 100

# # Read summa and NLDAS data
# print('read')
# # read summa 3-hourly P and T 
# sim_file='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/output/gmet_3hr/output_output_timestep.nc'
# p_vname = 'pptrate_mean' #mm/s 
# temp_vname = 'airtemp_mean' #'K' 
# t_vname = 'time'

# f = xr.open_dataset(sim_file)
# p_gmet = f[p_vname].values[:]
# temp_gmet = f[temp_vname].values[:]- 273.15 # K to C
# time_str_gmet = f[t_vname].dt.strftime(time_format).values[:]
# time_gmet = np.asarray([datetime.datetime.strptime(t, time_format) for t in time_str_gmet])
# f.close()

# # read hourly NLDAS P and T 
# sim_file='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/scripts_hongli/nldas_taylor_2007_2011.nc'
# p_vname = 'RAINRATE' #mm/s 
# temp_vname = 'T2D' #'K' 
# t_vname = 'valid_time'

# f = xr.open_dataset(sim_file)
# p_nldas = f[p_vname].values[:]
# temp_nldas = f[temp_vname].values[:]- 273.15 # K to C
# time_str_nldas = f[t_vname].dt.strftime(time_format).values[:]
# time_nldas = np.asarray([datetime.datetime.strptime(t, time_format) for t in time_str_nldas])
# f.close()

# # extract useful GMET and NLDAS data based on commonly available time and timestep (1 hour, 3 hours)
# gmet_time_end_index = np.where(time_gmet==time_nldas[-3])[0][0] #2011-12-31 21:00:00
# p_gmet_part = p_gmet[0:gmet_time_end_index+1,:]
# temp_gmet_part = temp_gmet[0:gmet_time_end_index+1,:]
# time_gmet_part = time_gmet[0:gmet_time_end_index+1]
# wateryear_list = np.unique([t.year for t in time_gmet_part])[1:]

# t_len = len(time_gmet_part)
# p_nldas_part = np.zeros((t_len,3))
# temp_nldas_part = np.zeros((t_len,3))
# for i in range(t_len):
#     t = time_gmet_part[i]
#     loc = np.where(time_nldas == t)[0]
#     p_nldas_part[i, :]=p_nldas[loc, :, 0]
#     temp_nldas_part[i, :]=temp_nldas[loc, :, 0]

# plot
print('plot')
os.chdir(os.path.join(root_dir, output_folder))
for hru in range(3):
    temp_fig_files = [] 
    print('hru '+str(hru+1))
    for m in range(len(wateryear_list)):

        # identify wateryear corresponding index
        wateryear = wateryear_list[m]
        wateryear_ini = datetime.datetime.strptime(str(wateryear-1)+'-10-01 00:00:00', time_format)
        wateryear_end = datetime.datetime.strptime(str(wateryear)+'-09-30 23:00:00', time_format)        
        wateryear_index=[loc for loc in range(len(time_gmet_part)) if (time_gmet_part[loc]>= wateryear_ini and time_gmet_part[loc]<=wateryear_end)]
        print(wateryear_ini)

        p_gmet_plot = p_gmet_part[wateryear_index,hru]
        temp_gmet_plot = temp_gmet_part[wateryear_index,hru]
        p_nldas_plot = p_nldas_part[wateryear_index,hru]
        temp_nldas_plot = temp_nldas_part[wateryear_index,hru]
        t_plot=[mpl.dates.date2num(hr) for hr in time_gmet_part[wateryear_index]]  

        temp_fig_file = 'image_'+str(m+1)+'.png'
        temp_fig_files.append(temp_fig_file)

        row_num = 2
        fig, ax = plt.subplots(row_num,1)
        fig.set_figwidth(6.5*1.3) #190mm
        fig.set_figheight(6.5*0.4*row_num)

        for k in range(row_num):
            subplot_left = 0.25
            subplot_base_bottom = (row_num-k)*1.0/row_num
            subplot_width = 0.7
            subplot_hight = 0.75/row_num        

                
            if k == 0: # plot P
                ax[k].set_position([subplot_left, subplot_base_bottom-subplot_hight-0.04, subplot_width, subplot_hight]) #[left, bottom, width, height]         
                ax[k].plot_date(t_plot, p_gmet_plot, 'b-', linewidth=0.75, markersize=0.0, alpha=0.8, label='GMET')  
                ax[k].plot_date(t_plot, p_nldas_plot, 'r:', linewidth=0.75, markersize=0.0, alpha=0.8, label='NLDAS')                
                ylabel = 'Precipitation \n (mm/s)'
                
                title_str = '('+chr(ord('a') + m) +') Water Year '+ str(wateryear)
    #             fig.suptitle(title_str, fontsize='medium', weight='semibold')
                fig.text(0.57, subplot_base_bottom-0.015, title_str, ha='center', va='center', rotation='horizontal', weight='semibold')

            else: # plot T
                ax[k].set_position([subplot_left, subplot_base_bottom-subplot_hight, subplot_width, subplot_hight]) #[left, bottom, width, height]         
                ax[k].plot_date(t_plot, temp_gmet_plot, 'b-', linewidth=0.75, markersize=0.0, alpha=0.8, label='GMET')  
                ax[k].plot_date(t_plot, temp_nldas_plot, 'r:', linewidth=0.75, markersize=0.0, alpha=0.8, label='NLDAS')
                ylabel = 'Temperature \n ($^\circ$C)'
            
            ax[k].xaxis.set_major_formatter(formatter)
            ax[k].xaxis.set_tick_params(labelsize='medium')#rotation=30, 
            fig.text(0.5*subplot_left, subplot_base_bottom-0.015-0.5*subplot_hight, ylabel, ha='center', va='center', rotation='horizontal')
            
            ax[k].legend(loc='upper left', fontsize='x-small', framealpha=0.5)



        fig.savefig(temp_fig_file, dpi=dpi_value)
        plt.close(fig)                    
        
    # concatenate into one figure    
    print('save')
    widths = []
    heights = []
    for temp_fig_file in temp_fig_files:
        im = Image.open(temp_fig_file)
        widths.append(im.width)
        heights.append(im.height)

    max_width = max(widths)
    total_height = sum(heights)
    new_im = Image.new('RGB', (max_width, total_height))

    x_offset = 0
    for temp_fig_file in temp_fig_files:
        im = Image.open(temp_fig_file)    
        new_im.paste(im, (0,x_offset))
        x_offset += im.size[1]
    new_im.save(outputfile)

    for file in os.listdir(os.getcwd()):
        if file.startswith('image'):
            os.remove(file)
    [os.remove(file) for file in os.getcwd() if file.startswith('image')]   

os.chdir(root_dir)


plot
hru 1
2007-10-01 00:00:00
2008-10-01 00:00:00
2009-10-01 00:00:00
2010-10-01 00:00:00
save
hru 2
2007-10-01 00:00:00
2008-10-01 00:00:00
2009-10-01 00:00:00
2010-10-01 00:00:00
save
hru 3
2007-10-01 00:00:00
2008-10-01 00:00:00
2009-10-01 00:00:00
2010-10-01 00:00:00
save


In [47]:
t_len = len(time_gmet_part)
p_nldas_part = np.zeros((t_len,3))
temp_nldas_part = np.zeros((t_len,3))
for i in range(t_len):
    t = time_gmet_part[i]
    loc = np.where(time_nldas == t)[0]
    p_nldas_part[i, :]=p_nldas[loc, :, 0]
    temp_nldas_part[i, :]=temp_nldas[loc, :, :]


In [48]:
np.shape(p_nldas_part)

(12424, 3)

In [33]:
sim_time_gmet

array([datetime.datetime(2007, 10, 1, 0, 0),
       datetime.datetime(2007, 10, 1, 3, 0),
       datetime.datetime(2007, 10, 1, 6, 0), ...,
       datetime.datetime(2017, 9, 29, 18, 0),
       datetime.datetime(2017, 9, 29, 21, 0),
       datetime.datetime(2017, 9, 30, 0, 0)], dtype=object)